<a href="https://colab.research.google.com/github/jdekle23/lite-llm-tool-agent/blob/main/ai_agent_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
"""
ai_agent.py

A minimal example of a tool‑using AI agent that can:
1. List files in the current working directory
2. Read the contents of a file
3. Terminate its own execution gracefully

The agent leverages LiteLLM's `completion` endpoint and demonstrates how to pass
structured tool definitions to an LLM, interpret the returned `tool_calls`, and
feed the tool responses back into the conversation loop.

Usage:
$ python ai_agent.py
> What would you like me to do? read the README
"""
!!pip install litellm

import json
import os
from typing import List, Dict, Callable

# --------------------------------------------------------------------------- #
#                  OPTIONAL: Pull API key from Google Colab userdata          #
# --------------------------------------------------------------------------- #
# These four lines let the script run in a Colab notebook where your OpenAI
# key is stored in `userdata`. If the import fails (e.g. you are *not* running
# inside Colab) the exception is silently ignored and the script falls back to
# whatever environment variable or auth method you normally use.

try:
    from google.colab import userdata  # type: ignore

    api_key = userdata.get("OPENAI_API_KEY")
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
except Exception:
    # ImportError (not Colab) or no key found — do nothing
    pass

from litellm import completion

In [14]:
# --------------------------------------------------------------------------- #
#                             Tool Implementations                            #
# --------------------------------------------------------------------------- #

def list_files() -> List[str]:
    """
    Return a list of file and directory names located in the current working
    directory.

    The function is intentionally simple because the agent—not a human—will
    parse the returned list and decide which file to read next.
    """
    return os.listdir(".")


def read_file(file_name: str) -> str:
    """
    Read and return the UTF‑8 text of *file_name*.

    Parameters
    ----------
    file_name : str
        The relative or absolute path to the file you want to read.

    Returns
    -------
    str
        The file contents if successful, otherwise a human‑readable error
        message that the agent can pass back to the user.
    """
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as exc:  # Catch‑all so the agent never crashes
        return f"Error: {str(exc)}"


def terminate(message: str) -> None:
    """
    Gracefully end the agent loop, printing *message* as a human‑friendly
    summary. In production you might return the message instead of printing.
    """
    print(f"Termination message: {message}")

    # Map each tool name to the underlying Python callable ---------------------- #
tool_functions: Dict[str, Callable] = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate,
}


In [15]:
# --------------------------------------------------------------------------- #
#                       OpenAI / LiteLLM Tool Descriptors                     #
# --------------------------------------------------------------------------- #
# These descriptors are sent as part of every LLM request so that the model
# understands the signature and purpose of each available function.

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the current directory.",
            # No parameters required
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": (
                "Stops the conversation loop and returns a final message to "
                "the user."
            ),
            "parameters": {
                "type": "object",
                "properties": {"message": {"type": "string"}},
                "required": ["message"],
            },
        },
    },
]

In [16]:
# --------------------------------------------------------------------------- #
#                               System Prompt                                 #
# --------------------------------------------------------------------------- #
agent_rules = [
    {
        "role": "system",
        "content": (
            "You are an AI agent that can perform tasks by invoking the tools "
            "I have provided.\n\n"
            "Workflow guidelines:\n"
            "1️⃣ If the user asks about files, *first* call list_files to show "
            "them what is available.\n"
            "2️⃣ Only after listing may you call read_file.\n"
            "3️⃣ When the task is complete, finish by calling terminate with a "
            "helpful summary."
        ),
    }
]


In [17]:
# --------------------------------------------------------------------------- #
#                            Agent Loop Parameters                            #
# --------------------------------------------------------------------------- #
iterations = 0               # Number of completed LLM calls
max_iterations = 10          # Safety valve to prevent runaway loops

user_task = input("What would you like me to do? ")
memory = [{"role": "user", "content": user_task}]


What would you like me to do? Which files do I have?


In [18]:
# --------------------------------------------------------------------------- #
#                                   Loop                                      #
# --------------------------------------------------------------------------- #
while iterations < max_iterations:
    iterations += 1
    messages = agent_rules + memory

    # Ask the LLM what to do next ------------------------------------------- #
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024,
    )

    # The model either calls a tool or replies directly --------------------- #
    tool_calls = response.choices[0].message.tool_calls
    if tool_calls:
        # For simplicity, we only execute the first tool call in this example
        tool = tool_calls[0]
        tool_name = tool.function.name
        tool_args = json.loads(tool.function.arguments)

        print(f"\n→ Executing: {tool_name}({tool_args})")

        if tool_name == "terminate":
            # terminate() handles its own printing
            terminate(**tool_args)
            break

        # Execute the Python function and capture its result ---------------- #
        try:
            result_payload = tool_functions[tool_name](**tool_args)
            result = {"result": result_payload}
        except Exception as exc:
            result = {"error": f"Error executing {tool_name}: {str(exc)}"}

        print(f"← Result: {result}")

        # Feed the tool invocation and the result back into the conversation #
        memory.extend(
            [
                {"role": "assistant", "content": json.dumps({"tool_name": tool_name, "args": tool_args})},
                {"role": "user", "content": json.dumps(result)},
            ]
        )
    else:
        # No tool call: the model sent a final natural‑language answer ------ #
        print(f"\nAssistant response: {response.choices[0].message.content}")
        break
else:
    # Loop exited because we hit max_iterations ----------------------------- #
    print("Max iterations reached without termination.")



→ Executing: list_files({})
← Result: {'result': ['.config', 'sample_data']}

Assistant response: It looks like you have a directory named `.config` and another named `sample_data`. If you need information about the contents of these directories or specific files, please let me know!
